# Variance et biais en fonction du budget

Tracer en fonction du budget :
- Variance ML
- Biais ML
- Variance LML
- Biais LML

In [ ]:
#import numpy as np
import cupy as np # Oui je sais c'est une abomination mais c'est juste pour pas à tout rechanger si on change de PC
import numpy as npp
import matplotlib.pyplot as plt
from tqdm import tqdm
np.fuse(npp.float32)

Définition des variables :

In [ ]:
def f_vec(X,v):
    return v.T@X

def f_vec_squared(X,v):
    return (v.T@X)**2

d = 10 # lenght of random vectors

v_1= np.array([-0.00606533, -0.02837768, -0.20481078, -0.05524456,  0.00408442, -0.02378791, -0.11289296, -0.09047946, -0.0828985,   0.01015773])
# eps = np.random.normal(0,0.001,d) 
# v_0 = v_1 + eps  #vecteur ceof basse fidélité 
# print(v_0)
# v_0 = np.array([-0.00589986, -0.02848255, -0.2058163,  -0.05332501,  0.00356343, -0.02399142, -0.11355733, -0.08986586, -0.08302782,  0.0085727 ])
v_0 = np.array([-0.04775578, -0.02158142, -0.22861181,  0.00999135 , 0.05261623, -0.03810132, -0.08892537, -0.09858141, -0.06851002,  0.09957945])
exp_var = np.sum(np.square(v_1))

print("Expected variance =", exp_var)

### Version Tenseur :

In [ ]:
# def f_vec_squared_tensor(X,v):
#     '''
#     Au final, fonction inutile car f_vec_squared s'adapte, c'est fou
#     '''
#     X_reorganized = np.transpose(X, (0,2,1))
    
#     result = np.einsum('ijk,k->ij', X_reorganized, v)
#     return result ** 2

In [ ]:
# d = 10
# M_0 = 17
# N = 10000
# X = np.random.standard_normal((N,d,M_0))
# X0 = np.random.standard_normal((N,d,M_0))

# # Y_bis = f_vec_squared_tensor(X,v_1)
# Y = f_vec_squared(X,v_1)
# Y0 = f_vec_squared(X,v_0)
# (Y-Y0).shape

# #np.var(np.mean(Y, axis=1))

Objectif : remplacer nos boucles for par des tenseurs pour direct faire des produits matriciels avec Numpy et ganger du temps de calcul

In [ ]:
def estim_var_MLMC(N,M_0,M_1):
    X1 = np.random.standard_normal((N,d,M_0))
    X0 = np.random.standard_normal((N,d,M_1))

    #estimateur de la variance 
    Y0_0_squared = f_vec_squared(X0,v_0)
    Y1_0_squared = f_vec_squared(X1,v_0)
    Y1_1_squared = f_vec_squared(X1,v_1)
    Var_MLMC = np.mean(Y0_0_squared, axis=1) + np.mean(Y1_1_squared-Y1_0_squared, axis=1)

    return Var_MLMC

def estim_var_MLMC_log(N,M_0,M_1):
    X1 = np.random.standard_normal((N,d,M_0))
    X0 = np.random.standard_normal((N,d,M_1))

    Y0_0_squared = f_vec_squared(X0,v_0)
    Y1_0_squared = f_vec_squared(X1,v_0)
    Y1_1_squared = f_vec_squared(X1,v_1)
    Var_MLMC_log = np.exp(np.log(np.mean(Y0_0_squared, axis=1)) + np.log(np.mean(Y1_1_squared, axis=1))-np.log(np.mean(Y1_0_squared, axis=1)))
    return Var_MLMC_log

# Variance et biais en fonction du budget :

#### Calcul des $M_\ell$ : 

Calcul de $M_\ell = m + 1 + \lfloor (\tilde{\eta} / \mathcal{S}_L) \sqrt{\mathcal{V}_\ell / \mathcal{C}_\ell} \rfloor > m $

avec : 
$$
 \tilde{\eta} = \eta - m \sum_{\ell' \le L} \mathcal{C}_{\ell'} \\
 \mathcal{S}_L = \sum_{\ell \le L} \sqrt{\mathcal{V}_\ell \mathcal{C}_\ell}
 $$


In [ ]:
def M_ell_tab(eta, V_ell, C_ell,m=0):
    """
    eta : budget
    V_ell : tableau contenant [V_0,V_1,...,V_L]
    C_ell : tableau contenant [C_0,C_1,...,C_L]

    return : la formule M_ell au dessus qui nous donne le nombre d'élement optimal à simuler
    """
    eta_tilde = eta # - m*np.sum(...)
    S_L = np.sum(np.sqrt(np.multiply(V_ell,C_ell)))

    return m + 1 + np.floor(
        (eta_tilde/S_L)*np.sqrt(V_ell*1/C_ell)
    )

##### Définition des constantes : 

Ici, nous avons :
$$
m = 0 \\
\mathcal{C}_1 = 1 \\
\mathcal{C}_0 = 0.5 \\
\eta = \text{varie de 10 à } 10^5
$$

De plus, on calcul nos $V_\ell$ avec un echantillon pilote de $N_{pilote} = 10000$.

Enfin, pour estimer le biais et la variance de nos estimateurs de la variance multi-level $V^{ML}_L$ et $V^{LogML}_L$, on prend une taille de $N = 10000$ (On génère $N$ valeurs de $V^{ML}_L$ et $V^{LogML}_L$ pour ensuite calculer leur variance et leur biais).

In [ ]:
N_pilote = 10000
X = np.random.standard_normal((d,N_pilote))
V_0 = np.var(f_vec_squared(X,v_0))
V_1 = np.var(f_vec_squared(X,v_1) - f_vec_squared(X,v_0))

V_ell = np.array([V_0,V_1])
C_ell = np.array([0.75,1.])

N=10000
budget_eta = npp.logspace(1,4,10)

In [ ]:
bias_tab = []
var_MLMC_tab = []
bias_log_tab = []
var_MLMC_log_tab = []

for eta in tqdm(budget_eta):
    # Pour chaque budget eta, calcul de nos M_0 et M_1
    M_0,M_1 = map(int, M_ell_tab(eta,V_ell,C_ell))

    # Calcul de nos tableaux de nos estimateurs de taille N
    Tab_var_MLMC = estim_var_MLMC(N,M_0,M_1)
    var_MLMC_tab.append(np.var(Tab_var_MLMC))
    bias_tab.append(np.square(np.mean(Tab_var_MLMC) - exp_var))

    Tab_var_MLMC = None
    np.get_default_memory_pool().free_all_blocks()

    Tab_var_MLMC_log = estim_var_MLMC_log(N,M_0,M_1)
    var_MLMC_log_tab.append(np.var(Tab_var_MLMC_log))
    bias_log_tab.append(np.square(np.mean(Tab_var_MLMC_log) - exp_var))

    Tab_var_MLMC_log = None
    np.get_default_memory_pool().free_all_blocks()

var_MLMC_tab = [x.item() for x in var_MLMC_tab]
bias_tab = [x.item() for x in bias_tab]

var_MLMC_log_tab = [x.item() for x in var_MLMC_log_tab]
bias_log_tab = [x.item() for x in bias_log_tab]

# Temps à battre avec deux boucle 'for' : 12m50
# avec les paramètres suivant :
# budget_eta = np.logspace(1,5,10)
# N=10000
    
# Temps à battre avec deux boucle 'for' : 1m37
# avec les paramètres suivant :
# budget_eta = np.logspace(1,4,10)
# N=10000

In [ ]:
# Plot log-log
plt.figure(figsize=(10, 6))

# Plot des données
plt.loglog(budget_eta, bias_tab, marker='o', label='Biais carré Tab')
plt.loglog(budget_eta, var_MLMC_tab, marker='s', label='Var MLMC Tab')
plt.loglog(budget_eta, bias_log_tab, marker='^', label='Biais carré Log Tab')
plt.loglog(budget_eta, var_MLMC_log_tab, marker='x', label='Var MLMC Log Tab')

# Ajouter titre et légendes
plt.title('Graphique Log-Log')
plt.xlabel('Budget eta')
plt.ylabel('Valeurs')
plt.legend()

# Afficher le plot
plt.grid(True)
plt.show()

In [ ]:
#np.get_default_memory_pool().free_all_blocks()

free_memory = np.cuda.Device().mem_info[0]
print("Mémoire libre sur le périphérique GPU :", free_memory)